In [1]:
import torch
import pyterrier as pt

if not pt.started():
    pt.init(tqdm="notebook")

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
dataset = pt.get_dataset("irds:beir/arguana")

In [5]:
bm25 = pt.BatchRetrieve("./beir_arguana", wmodel="BM25")

11:51:28.368 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


In [6]:
candidates = (~bm25 % 5)(dataset.get_topics())
candidates

,qid,docid,docno,rank,score,query
0,test-environment-aeghhgwpe-pro02a,1,test-environment-aeghhgwpe-pro02a,0,162.553590,being vegetarian helps the environment becomin...
1,test-environment-aeghhgwpe-pro02a,0,test-environment-aeghhgwpe-pro02b,1,66.622882,being vegetarian helps the environment becomin...
2,test-environment-aeghhgwpe-pro02a,7,test-environment-aeghhgwpe-pro04a,2,44.883224,being vegetarian helps the environment becomin...
3,test-environment-aeghhgwpe-pro02a,2,test-environment-aeghhgwpe-pro03b,3,40.830951,being vegetarian helps the environment becomin...
4,test-environment-aeghhgwpe-pro02a,1021,test-international-sepiahbaaw-pro02a,4,37.943781,being vegetarian helps the environment becomin...
...,...,...,...,...,...,...
1405000,test-society-epsihbdns-con01a,656,test-economy-epsihbdns-con01a,0,68.896176,freedom of movement is an intrinsic human righ...
1405001,test-society-epsihbdns-con01a,3561,training-free-speech-debate-ifdffehw-con04a,1,26.894490,freedom of movement is an intrinsic human righ...
1405002,test-society-epsihbdns-con01a,5505,training-philosophy-eppprfmhb-pro01a,2,25.027911,freedom of movement is an intrinsic human righ...
1405003,test-society-epsihbdns-con01a,4603,training-international-glhrsimhw-con04a,3,24.926126,freedom of movement is an intrinsic human righ...


### Artic-Embed

In [7]:
from fast_forward.encoder import TransformerEncoder
import torch

class SnowFlakeDocumentEncoder(TransformerEncoder):
  def __call__(self, texts):
    document_tokens =  self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    document_tokens.to(self.device)

    self.model.eval()
    # Compute token embeddings
    with torch.no_grad():
        #query_embeddings = self.model(**query_tokens)[0][:, 0]
        doument_embeddings = self.model(**document_tokens)[0][:, 0]

    # normalize embeddings
    #query_embeddings = torch.nn.functional.normalize(query_embeddings, p=2, dim=1)
    doument_embeddings = torch.nn.functional.normalize(doument_embeddings, p=2, dim=1)
    return doument_embeddings.detach().cpu().numpy()

In [8]:
class SnowFlakeQueryEncoder(TransformerEncoder):
  def __call__(self, texts):
    query_prefix = 'Represent this sentence for searching relevant passages: '
    queries_with_prefix = ["{}{}".format(query_prefix, i) for i in texts]
    query_tokens = self.tokenizer(queries_with_prefix, padding=True, truncation=True, return_tensors='pt', max_length=512)

    query_tokens.to(self.device)
    self.model.eval()
    #document_tokens =  self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    # Compute token embeddings
    with torch.no_grad():
        query_embeddings = self.model(**query_tokens)[0][:, 0]
        #doument_embeddings = self.model(**document_tokens)[0][:, 0]

    # normalize embeddings
    query_embeddings = torch.nn.functional.normalize(query_embeddings, p=2, dim=1)
    #doument_embeddings = torch.nn.functional.normalize(doument_embeddings, p=2, dim=1)
    return query_embeddings.detach().cpu().numpy()

In [9]:
doc_encoder_artic = SnowFlakeDocumentEncoder('Snowflake/snowflake-arctic-embed-m', device="cuda:0")
q_encoder_artic = SnowFlakeQueryEncoder('Snowflake/snowflake-arctic-embed-m')

/home/dodo/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dodo/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m and are newly i

In [10]:
from fast_forward import OnDiskIndex, Mode
from pathlib import Path

ff_index_artic = OnDiskIndex.load(
    Path("ffindex_arguana_snowflake.h5"), query_encoder=q_encoder_artic, mode=Mode.MAXP
)

  0%|          | 0/8674 [00:00<?, ?it/s]

100%|██████████| 8674/8674 [00:00<00:00, 1581593.40it/s]


In [11]:
ff_index_artic = ff_index_artic.to_memory()

In [12]:
from fast_forward.util.pyterrier import FFScore

ff_score_artic = FFScore(ff_index_artic)

In [13]:
re_ranked = ff_score_artic(candidates) # time it - do it separately from the experiments
re_ranked

,qid,docno,score_0,score,query
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,162.553590,0.926701,being vegetarian helps the environment becomin...
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,66.622882,0.738024,being vegetarian helps the environment becomin...
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,44.883224,0.493260,being vegetarian helps the environment becomin...
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,40.830951,0.552747,being vegetarian helps the environment becomin...
4,test-environment-aeghhgwpe-pro02a,test-international-sepiahbaaw-pro02a,37.943781,0.437614,being vegetarian helps the environment becomin...
...,...,...,...,...,...
7025,test-society-epsihbdns-con01a,test-economy-epsihbdns-con01a,68.896176,0.803223,freedom of movement is an intrinsic human righ...
7026,test-society-epsihbdns-con01a,training-free-speech-debate-ifdffehw-con04a,26.894490,0.534158,freedom of movement is an intrinsic human righ...
7027,test-society-epsihbdns-con01a,training-philosophy-eppprfmhb-pro01a,25.027911,0.534102,freedom of movement is an intrinsic human righ...
7028,test-society-epsihbdns-con01a,training-international-glhrsimhw-con04a,24.926126,0.414428,freedom of movement is an intrinsic human righ...


In [14]:
from fast_forward.util.pyterrier import FFInterpolate

alpha_values = [0.05, 0.1, 0.3, 0.5, 0.8]
dataframes = []

## Trying to use different alpha values 
for alpha in alpha_values:
    ff_int = FFInterpolate(alpha=alpha)
    dataframes.append(ff_int(re_ranked))

In [15]:
ff_int2 = FFInterpolate(alpha=0.1)
ff_int2(re_ranked)

,qid,docno,query,score
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,being vegetarian helps the environment becomin...,17.089390
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,being vegetarian helps the environment becomin...,7.326510
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,being vegetarian helps the environment becomin...,4.932257
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,being vegetarian helps the environment becomin...,4.580568
4,test-environment-aeghhgwpe-pro02a,test-international-sepiahbaaw-pro02a,being vegetarian helps the environment becomin...,4.188231
...,...,...,...,...
7025,test-society-epsihbdns-con01a,test-economy-epsihbdns-con01a,freedom of movement is an intrinsic human righ...,7.612518
7026,test-society-epsihbdns-con01a,training-free-speech-debate-ifdffehw-con04a,freedom of movement is an intrinsic human righ...,3.170191
7027,test-society-epsihbdns-con01a,training-philosophy-eppprfmhb-pro01a,freedom of movement is an intrinsic human righ...,2.983483
7028,test-society-epsihbdns-con01a,training-international-glhrsimhw-con04a,freedom of movement is an intrinsic human righ...,2.865598


In [16]:
dataframes[1] ## This is the dataframe for alpha=0.1 - same as the one above

,qid,docno,query,score
0,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02a,being vegetarian helps the environment becomin...,17.089390
1,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro02b,being vegetarian helps the environment becomin...,7.326510
2,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro04a,being vegetarian helps the environment becomin...,4.932257
3,test-environment-aeghhgwpe-pro02a,test-environment-aeghhgwpe-pro03b,being vegetarian helps the environment becomin...,4.580568
4,test-environment-aeghhgwpe-pro02a,test-international-sepiahbaaw-pro02a,being vegetarian helps the environment becomin...,4.188231
...,...,...,...,...
7025,test-society-epsihbdns-con01a,test-economy-epsihbdns-con01a,freedom of movement is an intrinsic human righ...,7.612518
7026,test-society-epsihbdns-con01a,training-free-speech-debate-ifdffehw-con04a,freedom of movement is an intrinsic human righ...,3.170191
7027,test-society-epsihbdns-con01a,training-philosophy-eppprfmhb-pro01a,freedom of movement is an intrinsic human righ...,2.983483
7028,test-society-epsihbdns-con01a,training-international-glhrsimhw-con04a,freedom of movement is an intrinsic human righ...,2.865598


In [17]:
from pyterrier.measures import RR, nDCG, MAP

pt.Experiment(
    [~bm25 % 1000 >> ff_score_artic >> dataframes[1]],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25 >> FF"],
  )

/tmp/ipykernel_751/1759268779.py:4: DeprecationWarning: Coercion of a dataframe into a transformer is deprecated; use a pt.Transformer.from_df() instead
  [~bm25 % 1000 >> ff_score_artic >> dataframes[1]],


,name,RR@10,nDCG@10,AP@100
0,BM25 >> FF,0.217544,0.305525,0.217579


In [18]:
pt.Experiment(
    [~bm25 % 1000 >> ff_score_artic >> ff_int2],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=["BM25 >> FF"],
  )

,name,RR@10,nDCG@10,AP@100
0,BM25 >> FF,0.246913,0.374217,0.257862
